In [15]:
import numpy as np
import matplotlib.pyplot as plt
from ORRFA import *
import interpretableai
from interpretableai import iai
import pandas as pd
import json
from HR import *

In [16]:
dataset = pd.read_csv("/Users/ryanlucas/Desktop/kc_house_data.csv")

In [64]:
dataset

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,263000018,20140521T000000,360000.0,3,2.50,1530,1131,3.0,0,0,...,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
21609,6600060120,20150223T000000,400000.0,4,2.50,2310,5813,2.0,0,0,...,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
21610,1523300141,20140623T000000,402101.0,2,0.75,1020,1350,2.0,0,0,...,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
21611,291310100,20150116T000000,400000.0,3,2.50,1600,2388,2.0,0,0,...,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287


In [ ]:
train_idx = np.arange(0, int(0.2*len(dataset)))
test_idx = np.arange(int(0.2*len(dataset)), int(0.3*len(dataset)))

features = dataset.loc[train_idx, ["floors", "bedrooms",
    "bathrooms",
    "sqft_above",
    "sqft_basement",
    "yr_built",
    "yr_renovated",
    "sqft_living",
    "sqft_lot",
    "lat",
    "long",
    "sqft_living15",
    "sqft_lot15"]]

diagnosis = dataset.loc[train_idx, "price"]

orig_features_size = features.copy().shape[1]

In [ ]:
features_test = dataset.loc[test_idx, ["floors", "bedrooms",
    "bathrooms",
    "sqft_above",
    "sqft_basement",
    "yr_built",
    "yr_renovated",
    "sqft_living",
    "sqft_lot",
    "lat",
    "long",
    "sqft_living15",
    "sqft_lot15"]]
diagnosis_test = dataset.loc[test_idx, 'price']

In [ ]:
diagnosis

In [ ]:
print(len(dataset))
print(len(features))
print(len(features_test))

In [ ]:
features

In [ ]:
def gen_features(sub_paths, data, features, orig_feature_size):
    new_features = features.copy()
    for path in sub_paths:
        statement = gen_statement(path, data, features)
        if statement != '(0 < 0.0)' and statement != '(0 >= 0.0)':

            try:
                index_feature = features.loc[eval(statement)].index


                new_feature = [1 if i in index_feature else 0 for i in range(len(features))]

                if sum(new_feature) >= 1:
                    new_features[statement] = new_feature

            except:
                print(statement)
    return new_features.iloc[:, orig_feature_size:]

In [ ]:
np.random.seed(1)
n = 5
rules = {}
for i in range(1,15):

    #################
    sampled_features = features.copy().sample(n=int(n),axis='columns')

    OCT_H = iai.GridSearch(
          iai.OptimalTreeRegressor(
              max_depth = 3, cp = 0.00000000001))

    OCT_H.fit(sampled_features, diagnosis)
    OCT_H.write_json(f"tre_{i}.json")

    ######################
    f = open(f"tre_{i}.json")
    data = json.load(f)

    paths = gen_paths(data)

    rule_features_new = gen_features(
            paths, data, features, orig_features_size)

    rules[f"tre_{i}"] = rule_features_new

    print(f"Tree: {i}")


In [11]:
OCT_H.get_learner()

Tree: 25
Tree: 26
Tree: 27
Tree: 28
Tree: 29
Tree: 30
Tree: 31
Tree: 32
Tree: 33
Tree: 34
Tree: 35
Tree: 36
Tree: 37
Tree: 38
Tree: 39
Tree: 40
Tree: 41
Tree: 42
Tree: 43
Tree: 44
Tree: 45
Tree: 46
Tree: 47
Tree: 48
Tree: 49


In [41]:
features.columns

Index(['Intercept', 'floors', 'bedrooms', 'bathrooms', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'sqft_living', 'sqft_lot',
       'lat', 'long', 'sqft_living15', 'sqft_lot15',
       '(features['long'] >= -0.5318101220723337)',
       '(features['long'] >= -0.5318101220723337) & (features['bathrooms'] < 7.503080016073889)',
       '(features['long'] >= -0.5318101220723337) & (features['bathrooms'] < 7.503080016073889) & (0 >= 0.0)',
       '(features['yr_built'] < -0.9539948558207259)',
       '(features['yr_built'] < -0.9539948558207259) & (features['long'] < -0.38450789661067586)',
       '(features['yr_built'] < -0.9539948558207259) & (features['long'] < -0.38450789661067586) & (0 >= 0.0)',
       '(features['sqft_above'] < -0.40000227641001507)',
       '(features['sqft_above'] < -0.40000227641001507) & (features['bathrooms'] < -0.5724146753574244)',
       '(features['sqft_above'] < -0.40000227641001507) & (features['bathrooms'] < -0.5724146753574244) & (0

In [42]:
data['lnr']['tree_']['nodes'][11]

{'id': 12,
 'parent': 11,
 'lower_child': -2,
 'upper_child': -2,
 'depth': 4,
 'split_type': 'LEAF',
 'split_mixed': {'parallel_split': {'feature': 0, 'threshold': 0.0},
  'ordinal_split': {'feature': 0, 'split': []},
  'categoric_split': {'feature': 0, 'split': [False]},
  'do_parallel': False,
  'do_ordinal': False},
 'split_hyperplane': {'standardized_weights': {'n': 5,
   'nzval': [],
   'nzind': []},
  'threshold': 0.0,
  'weights': {'n': 5, 'nzval': [], 'nzind': []},
  'complexity': 0.0},
 'missingdatamode': 'NONE',
 'raw_error': 150705677230466.4,
 'baseline_error': 150705677230466.4,
 'subtree_complexity': 0.0,
 'n_node_samples': 3227,
 'weighted_n_node_samples': 3227.0,
 'orig_raw_error': 150705677230466.4,
 'orig_baseline_error': 150705677230466.4,
 'dirty': True,
 'fit': {'beta': {'n': 5, 'nzval': [], 'nzind': []},
  'offset': 523524.58723272354,
  'standardized_beta': {'n': 5, 'nzval': [], 'nzind': []},
  'constant_offset': 523524.58723272354,
  'constant_error': 150705677

In [43]:
features.columns

Index(['Intercept', 'floors', 'bedrooms', 'bathrooms', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'sqft_living', 'sqft_lot',
       'lat', 'long', 'sqft_living15', 'sqft_lot15',
       '(features['long'] >= -0.5318101220723337)',
       '(features['long'] >= -0.5318101220723337) & (features['bathrooms'] < 7.503080016073889)',
       '(features['long'] >= -0.5318101220723337) & (features['bathrooms'] < 7.503080016073889) & (0 >= 0.0)',
       '(features['yr_built'] < -0.9539948558207259)',
       '(features['yr_built'] < -0.9539948558207259) & (features['long'] < -0.38450789661067586)',
       '(features['yr_built'] < -0.9539948558207259) & (features['long'] < -0.38450789661067586) & (0 >= 0.0)',
       '(features['sqft_above'] < -0.40000227641001507)',
       '(features['sqft_above'] < -0.40000227641001507) & (features['bathrooms'] < -0.5724146753574244)',
       '(features['sqft_above'] < -0.40000227641001507) & (features['bathrooms'] < -0.5724146753574244) & (0

In [44]:
big_df = pd.DataFrame()
for tree in rules.keys():

    big_df = pd.concat([big_df, rules[tree]],axis =1)


In [45]:
df = big_df.loc[:,~big_df.columns.duplicated()].copy()

In [46]:
df

,(features['long'] >= -0.5318101220723337),(features['long'] >= -0.5318101220723337) & (features['bathrooms'] < 7.503080016073889),(features['long'] >= -0.5318101220723337) & (features['bathrooms'] < 7.503080016073889) & (0 >= 0.0),(features['yr_built'] < -0.9539948558207259),(features['yr_built'] < -0.9539948558207259) & (features['long'] < -0.38450789661067586),(features['yr_built'] < -0.9539948558207259) & (features['long'] < -0.38450789661067586) & (0 >= 0.0),(features['sqft_above'] < -0.40000227641001507),(features['sqft_above'] < -0.40000227641001507) & (features['bathrooms'] < -0.5724146753574244),(features['sqft_above'] < -0.40000227641001507) & (features['bathrooms'] < -0.5724146753574244) & (0 >= 0.0),(features['bathrooms'] >= 2.723705606859437),...,(features['sqft_living'] >= 4.531003639864835) & (features['sqft_lot'] >= 0.8077444289047583),(features['sqft_living'] >= 4.531003639864835) & (features['sqft_lot'] >= 0.8077444289047583) & (0 >= 0.0),(features['long'] < -0.18331461305323282),(features['long'] < -0.18331461305323282) & (features['long'] < -1.2108374540785394),(features['sqft_living'] < 0.8988702333203695),(features['sqft_living'] < 0.8988702333203695) & (features['lat'] >= -0.23697575021859496),(features['sqft_living'] >= 4.531003639864835) & (features['lat'] < -0.24162683648000938),(features['sqft_living15'] >= 2.631907439115192),(features['sqft_living15'] >= 2.631907439115192) & (features['sqft_living15'] >= 3.9241478038405315),(features['sqft_living15'] >= 2.631907439115192) & (features['sqft_living15'] >= 3.9241478038405315) & (0 >= 0.0)
0,1,1,1,0,0,0,1,1,1,0,...,0,0,1,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,1,0,0,0,0
2,1,1,1,1,0,0,1,1,1,0,...,0,0,0,0,1,1,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,1,1,1,0,0,0,0,0
4,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15124,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
15125,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15126,1,1,1,0,0,0,0,0,0,0,...,0,0,1,0,1,1,0,0,0,0
15127,0,0,0,0,0,0,1,1,1,0,...,0,0,1,0,1,1,0,0,0,0


In [47]:
for column in df.columns[1:]:
    if df[column].sum() == len(df):
        del df[column]


In [48]:
df

,(features['long'] >= -0.5318101220723337),(features['long'] >= -0.5318101220723337) & (features['bathrooms'] < 7.503080016073889),(features['long'] >= -0.5318101220723337) & (features['bathrooms'] < 7.503080016073889) & (0 >= 0.0),(features['yr_built'] < -0.9539948558207259),(features['yr_built'] < -0.9539948558207259) & (features['long'] < -0.38450789661067586),(features['yr_built'] < -0.9539948558207259) & (features['long'] < -0.38450789661067586) & (0 >= 0.0),(features['sqft_above'] < -0.40000227641001507),(features['sqft_above'] < -0.40000227641001507) & (features['bathrooms'] < -0.5724146753574244),(features['sqft_above'] < -0.40000227641001507) & (features['bathrooms'] < -0.5724146753574244) & (0 >= 0.0),(features['bathrooms'] >= 2.723705606859437),...,(features['sqft_living'] >= 4.531003639864835) & (features['sqft_lot'] >= 0.8077444289047583),(features['sqft_living'] >= 4.531003639864835) & (features['sqft_lot'] >= 0.8077444289047583) & (0 >= 0.0),(features['long'] < -0.18331461305323282),(features['long'] < -0.18331461305323282) & (features['long'] < -1.2108374540785394),(features['sqft_living'] < 0.8988702333203695),(features['sqft_living'] < 0.8988702333203695) & (features['lat'] >= -0.23697575021859496),(features['sqft_living'] >= 4.531003639864835) & (features['lat'] < -0.24162683648000938),(features['sqft_living15'] >= 2.631907439115192),(features['sqft_living15'] >= 2.631907439115192) & (features['sqft_living15'] >= 3.9241478038405315),(features['sqft_living15'] >= 2.631907439115192) & (features['sqft_living15'] >= 3.9241478038405315) & (0 >= 0.0)
0,1,1,1,0,0,0,1,1,1,0,...,0,0,1,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,1,0,0,0,0
2,1,1,1,1,0,0,1,1,1,0,...,0,0,0,0,1,1,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,1,1,1,0,0,0,0,0
4,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15124,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
15125,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15126,1,1,1,0,0,0,0,0,0,0,...,0,0,1,0,1,1,0,0,0,0
15127,0,0,0,0,0,0,1,1,1,0,...,0,0,1,0,1,1,0,0,0,0


In [49]:
features_for_robust_reg = df

In [50]:
rule_features = features_for_robust_reg.copy()
features_for_robust_reg = pd.concat([features, features_for_robust_reg], axis = 1)

In [51]:
features_for_robust_reg.sum()

Intercept                                                                                                            6.484000e+03
floors                                                                                                              -2.273737e-13
bedrooms                                                                                                             1.065814e-12
bathrooms                                                                                                            0.000000e+00
sqft_above                                                                                                           4.547474e-13
                                                                                                                         ...     
(features['sqft_living'] < 0.8988702333203695) & (features['lat'] >= -0.23697575021859496)                           7.788000e+03
(features['sqft_living'] >= 4.531003639864835) & (features['lat'] < -0.24162683648000938) 

In [52]:
features_for_robust_reg.insert(loc = 0, column = 'Intercept', value = [1 for i in range(len(features_for_robust_reg))])

ValueError: cannot insert Intercept, already exists

In [53]:
features_for_robust_reg

,Intercept,floors,bedrooms,bathrooms,sqft_above,sqft_basement,yr_built,yr_renovated,sqft_living,sqft_lot,...,(features['sqft_living'] >= 4.531003639864835) & (features['sqft_lot'] >= 0.8077444289047583),(features['sqft_living'] >= 4.531003639864835) & (features['sqft_lot'] >= 0.8077444289047583) & (0 >= 0.0),(features['long'] < -0.18331461305323282),(features['long'] < -0.18331461305323282) & (features['long'] < -1.2108374540785394),(features['sqft_living'] < 0.8988702333203695),(features['sqft_living'] < 0.8988702333203695) & (features['lat'] >= -0.23697575021859496),(features['sqft_living'] >= 4.531003639864835) & (features['lat'] < -0.24162683648000938),(features['sqft_living15'] >= 2.631907439115192),(features['sqft_living15'] >= 2.631907439115192) & (features['sqft_living15'] >= 3.9241478038405315),(features['sqft_living15'] >= 2.631907439115192) & (features['sqft_living15'] >= 3.9241478038405315) & (0 >= 0.0)
0,1.0,-0.239863,0.619419,0.646855,-0.201145,1.435266,-2.056083,-0.192198,0.457977,-0.256426,...,0,0,1,0,1,0,0,0,0,0
1,1.0,0.623698,0.619419,0.326955,1.513861,-0.622013,0.593992,-0.192198,1.114952,-0.025031,...,0,0,1,0,1,1,0,0,0,0
2,1.0,-1.103424,-0.404890,-1.592446,-1.202525,-0.622013,-1.205442,-0.192198,-1.385790,-0.271136,...,0,0,0,0,1,1,0,0,0,0
3,1.0,-1.103424,0.619419,0.326955,0.155668,1.624441,-0.354801,-0.192198,0.871235,-0.140820,...,0,0,1,1,1,0,0,0,0,0
4,1.0,-1.103424,-1.429198,-1.592446,-1.260076,-0.622013,-0.976423,-0.192198,-1.438772,-0.152930,...,0,0,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,1,1,0,0,0,0
15125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
15126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,1,0,1,1,0,0,0,0
15127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,1,0,1,1,0,0,0,0


In [54]:
features_for_robust_reg

,Intercept,floors,bedrooms,bathrooms,sqft_above,sqft_basement,yr_built,yr_renovated,sqft_living,sqft_lot,...,(features['sqft_living'] >= 4.531003639864835) & (features['sqft_lot'] >= 0.8077444289047583),(features['sqft_living'] >= 4.531003639864835) & (features['sqft_lot'] >= 0.8077444289047583) & (0 >= 0.0),(features['long'] < -0.18331461305323282),(features['long'] < -0.18331461305323282) & (features['long'] < -1.2108374540785394),(features['sqft_living'] < 0.8988702333203695),(features['sqft_living'] < 0.8988702333203695) & (features['lat'] >= -0.23697575021859496),(features['sqft_living'] >= 4.531003639864835) & (features['lat'] < -0.24162683648000938),(features['sqft_living15'] >= 2.631907439115192),(features['sqft_living15'] >= 2.631907439115192) & (features['sqft_living15'] >= 3.9241478038405315),(features['sqft_living15'] >= 2.631907439115192) & (features['sqft_living15'] >= 3.9241478038405315) & (0 >= 0.0)
0,1.0,-0.239863,0.619419,0.646855,-0.201145,1.435266,-2.056083,-0.192198,0.457977,-0.256426,...,0,0,1,0,1,0,0,0,0,0
1,1.0,0.623698,0.619419,0.326955,1.513861,-0.622013,0.593992,-0.192198,1.114952,-0.025031,...,0,0,1,0,1,1,0,0,0,0
2,1.0,-1.103424,-0.404890,-1.592446,-1.202525,-0.622013,-1.205442,-0.192198,-1.385790,-0.271136,...,0,0,0,0,1,1,0,0,0,0
3,1.0,-1.103424,0.619419,0.326955,0.155668,1.624441,-0.354801,-0.192198,0.871235,-0.140820,...,0,0,1,1,1,0,0,0,0,0
4,1.0,-1.103424,-1.429198,-1.592446,-1.260076,-0.622013,-0.976423,-0.192198,-1.438772,-0.152930,...,0,0,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,1,1,0,0,0,0
15125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
15126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,1,0,1,1,0,0,0,0
15127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,1,0,1,1,0,0,0,0


In [55]:
model = HolisticRobust(
                       α = 0,
                       ϵ = 0,
                       r = 0.05,
                       classifier = "Linear",
                       learning_approach = "HR") # Could be ERM either

In [56]:
θ, obj_value = model.fit(ξ = (np.matrix(features_for_robust_reg), np.array(diagnosis)))

KeyboardInterrupt: 

In [ ]:
predictions = np.multiply(θ, features_for_robust_reg
                          ).sum(axis = 1)

In [ ]:
diagnosis.plot()

In [ ]:
(diagnosis - predictions).describe()

In [ ]:
SSE_model = sum((diagnosis - predictions)**2)
SSE_mean = sum((diagnosis - diagnosis.mean())**2)

1 - SSE_model/SSE_mean

In [32]:
features

-1.256205050439164e+23

In [33]:
rule_features

In [34]:
features = features_test

for rule in rule_features.columns:

    index_feature = features.loc[eval(rule)].index

    new_feature = [1 if i in index_feature else 0 for i in range(len(features))]

    if sum(new_feature) >= 1:
        features[rule] = new_feature


In [35]:
new_features = features_test

In [57]:
new_features.insert(loc = 0, column = 'Intercept', value = [1 for i in range(len(new_features))])

ValueError: cannot insert Intercept, already exists

In [58]:
predictions_test = lasso_trees.predict(features_test)